**Machine Learning-Based Analysis of the 2023 Morocco Earthquake Using Maxar Satellite Imagery**

This notebook demonstrates how to use Maxar Open Data satellite imagery to perform automated damage assessment using deep learning models. We'll use pre- and post-earthquake imagery to detect and classify building damage.

The analysis combines:
- Maxar high-resolution satellite imagery
- Building footprint detection
- Change detection
- Damage classification

In [ ]:
# Install required packages
!pip install -U leafmap geopandas cogeo-mosaic torch torchvision segmentation-models-pytorch rasterio numpy pandas scikit-learn opencv-python

  Using cached leafmap-0.42.6-py2.py3-none-any.whl (513 kB)
  Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
  Using cached cogeo_mosaic-8.0.0-py3-none-any.whl (40 kB)
     ------------------------------------ 203.1/203.1 MB 990.3 kB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 1.3 MB/s eta 0:00:00
  Using cached segmentation_models_pytorch-0.4.0-py3-none-any.whl (121 kB)
     ---------------------------------------- 25.4/25.4 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 12.9/12.9 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 11.6/11.6 MB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 11.1/11.1 MB 1.1 MB/s eta 0:00:00
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
  Using cached colour-0.1.5-py2.py3-none-any.whl (23 kB)
  Using cached ipyleaflet-0.19.2-py3-none-any.whl (31 kB)
  Using cached pystac_client-0.8.5-py3-none-any.whl (41 kB)
  Using cached ipyfilec

ERROR: Exception:
Traceback (most recent call last):
  File "c:\Users\mohamed\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "c:\Users\mohamed\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "c:\Users\mohamed\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "c:\Users\mohamed\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "c:\Users\mohamed\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 465, in read
    s = self.fp.read(amt)
  File "c:\Users\mohamed\AppData\Local\Programs\Python\Python310\lib\socket.py", line 705

In [ ]:
# Install missing packages
import leafmap.foliumap as leafmap
import torch
import torchvision
import segmentation_models_pytorch as smp
import rasterio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2

### 1. Data Preparation

In [ ]:
def load_and_preprocess_image(url):
    """Load and preprocess satellite image from URL"""
    with rasterio.open(url) as src:
        img = src.read([1,2,3])  # Read RGB bands
        img = np.moveaxis(img, 0, -1)  # Convert to HWC format
        img = cv2.resize(img, (512, 512))  # Resize to consistent size
        img = img.astype(np.float32) / 255.0  # Normalize to [0,1]
    return img

In [ ]:
# Get pre and post event imagery
gdf = leafmap.maxar_items(
    collection_id="Morocco-Earthquake-Sept-2023",
    child_id="10300100ECC53700",
    return_gdf=True,
    assets=["visual"]
)

before_images = gdf[gdf["datetime"] < "2023-09-10"]["visual"].tolist()
after_images = gdf[gdf["datetime"] >= "2023-09-10"]["visual"].tolist()

### 2. Building Detection Model

In [ ]:
class BuildingDetectionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = smp.Unet(
            encoder_name="resnet34",
            encoder_weights="imagenet",
            in_channels=3,
            classes=1,
        )
    
    def forward(self, x):
        return self.model(x)

building_detector = BuildingDetectionModel()
# In practice, you would load pre-trained weights here
# building_detector.load_state_dict(torch.load('building_detector_weights.pth'))

### 3. Change Detection Model

In [ ]:
class ChangeDetectionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Siamese network architecture
        self.encoder = torchvision.models.resnet34(pretrained=True)
        self.encoder.fc = torch.nn.Identity()  # Remove classification head
        
        self.change_head = torch.nn.Sequential(
            torch.nn.Linear(512 * 2, 256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256, 1),
            torch.nn.Sigmoid()
        )
    
    def forward(self, x1, x2):
        # Extract features from both images
        feat1 = self.encoder(x1)
        feat2 = self.encoder(x2)
        # Concatenate and compute change score
        combined = torch.cat([feat1, feat2], dim=1)
        return self.change_head(combined)

change_detector = ChangeDetectionModel()
# change_detector.load_state_dict(torch.load('change_detector_weights.pth'))

### 4. Damage Classification

In [ ]:
class DamageClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torchvision.models.resnet50(pretrained=True)
        self.model.fc = torch.nn.Linear(2048, 4)  # 4 damage levels
        
    def forward(self, x):
        return self.model(x)

damage_classifier = DamageClassifier()
# damage_classifier.load_state_dict(torch.load('damage_classifier_weights.pth'))

### 5. Complete Analysis Pipeline

In [ ]:
def analyze_damage(before_url, after_url):
    """Complete pipeline for damage analysis"""
    # Load and preprocess images
    before_img = load_and_preprocess_image(before_url)
    after_img = load_and_preprocess_image(after_url)
    
    # Convert to torch tensors
    before_tensor = torch.from_numpy(before_img).permute(2,0,1).unsqueeze(0)
    after_tensor = torch.from_numpy(after_img).permute(2,0,1).unsqueeze(0)
    
    # Detect buildings
    with torch.no_grad():
        building_mask = building_detector(after_tensor)
        
        # Detect changes
        change_score = change_detector(before_tensor, after_tensor)
        
        # Classify damage for detected buildings
        damage_pred = damage_classifier(after_tensor)
    
    return {
        'building_mask': building_mask.squeeze().numpy(),
        'change_score': change_score.item(),
        'damage_class': torch.argmax(damage_pred).item()
    }

### 6. Visualization Functions

In [ ]:
def visualize_results(before_url, after_url, results):
    """Create visualization of analysis results"""
    damage_classes = ['No Damage', 'Minor', 'Major', 'Destroyed']
    
    m = leafmap.Map()
    
    # Add before/after imagery
    m.split_map(before_url, after_url)
    
    # Overlay building detection mask
    m.add_raster(results['building_mask'], layer_name='Buildings')
    
    # Add damage classification results
    damage_level = damage_classes[results['damage_class']]
    m.add_text(f"Damage Level: {damage_level}")
    m.add_text(f"Change Score: {results['change_score']:.2f}")
    
    return m

### 7. Run Analysis

In [ ]:
# Analyze sample image pair
sample_before = before_images[0]
sample_after = after_images[0]

results = analyze_damage(sample_before, sample_after)
m = visualize_results(sample_before, sample_after, results)
m

### 8. Batch Processing

In [ ]:
def batch_analyze_region(before_images, after_images):
    """Analyze multiple image pairs and aggregate results"""
    results = []
    
    for before_url, after_url in zip(before_images, after_images):
        try:
            analysis = analyze_damage(before_url, after_url)
            results.append({
                'before_url': before_url,
                'after_url': after_url,
                **analysis
            })
        except Exception as e:
            print(f"Error processing {before_url}: {str(e)}")
    
    return pd.DataFrame(results)

In [ ]:
# Run batch analysis
results_df = batch_analyze_region(before_images[:5], after_images[:5])
results_df.head()